In [1]:
import pandas as pd
import numpy as np
import boto3

In [2]:
s3=boto3.client("s3")
bucket_name="loanpredictiondataset"
file_key="cleaned_data.csv"
obj=s3.get_object(Bucket=bucket_name,Key=file_key)
data=pd.read_csv(obj["Body"])

In [3]:
data

,Unnamed: 0,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,0,Male,No,0,Graduate,No,5849.0,0.0,146.412162,360.0,1.0,Urban,yes
1,1,Male,Yes,1,Graduate,No,4583.0,1508.0,128.000000,360.0,1.0,Rural,no
2,2,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.000000,360.0,1.0,Urban,yes
3,3,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.000000,360.0,1.0,Urban,yes
4,4,Male,No,0,Graduate,No,6000.0,0.0,141.000000,360.0,1.0,Urban,yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,609,Female,No,0,Graduate,No,2900.0,0.0,71.000000,360.0,1.0,Rural,yes
610,610,Male,Yes,3,Graduate,No,4106.0,0.0,40.000000,180.0,1.0,Rural,yes
611,611,Male,Yes,1,Graduate,No,8072.0,240.0,253.000000,360.0,1.0,Urban,yes
612,612,Male,Yes,2,Graduate,No,7583.0,0.0,187.000000,360.0,1.0,Urban,yes


In [4]:
data["Dependents"]=data["Dependents"].astype("object")

In [5]:
data=data.drop("Unnamed: 0",axis=1)

# lets do feature creation

In [6]:
data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status
0,Male,No,0,Graduate,No,5849.0,0.0,146.412162,360.0,1.0,Urban,yes
1,Male,Yes,1,Graduate,No,4583.0,1508.0,128.000000,360.0,1.0,Rural,no
2,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.000000,360.0,1.0,Urban,yes
3,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.000000,360.0,1.0,Urban,yes
4,Male,No,0,Graduate,No,6000.0,0.0,141.000000,360.0,1.0,Urban,yes
...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900.0,0.0,71.000000,360.0,1.0,Rural,yes
610,Male,Yes,3,Graduate,No,4106.0,0.0,40.000000,180.0,1.0,Rural,yes
611,Male,Yes,1,Graduate,No,8072.0,240.0,253.000000,360.0,1.0,Urban,yes
612,Male,Yes,2,Graduate,No,7583.0,0.0,187.000000,360.0,1.0,Urban,yes


In [7]:
col=data.describe().columns

In [8]:
list(col)

['ApplicantIncome',
 'CoapplicantIncome',
 'LoanAmount',
 'Loan_Amount_Term',
 'Credit_History']

# first lets scheck if data is skewe or not

In [9]:
list_skew=[]
for i in list(col):
    skew=data[i].skew() 
    list_skew.append(skew)
    

In [10]:
list_skew

[6.539513113994625,
 7.491531216666236,
 2.726601144105299,
 -2.3896799467122745,
 -1.963600485467902]

# so now ApplicantIncome','CoapplicantIncome','LoanAmount' these show strong positive skew we need to log traansform

In [11]:
data["Applicant_log"]=np.log1p(data["ApplicantIncome"])
data["CoapplicantIncome_log"]=np.log1p(data["CoapplicantIncome"])
data["LoanAmount_log"]=np.log1p(data["LoanAmount"])

# now we do binning

In [12]:
data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Applicant_log,CoapplicantIncome_log,LoanAmount_log
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,5403.459283,1621.245798,146.412162,342.000000,0.842199,8.341510,4.290129,4.870872
std,6109.041673,2926.248369,84.037468,64.372489,0.349681,0.645035,3.875873,0.491376
min,150.000000,0.000000,9.000000,12.000000,0.000000,5.017280,0.000000,2.302585
25%,2877.500000,0.000000,100.250000,360.000000,1.000000,7.965024,0.000000,4.617584
50%,3812.500000,1188.500000,129.000000,360.000000,1.000000,8.246303,7.081125,4.867534
75%,5795.000000,2297.250000,164.750000,360.000000,1.000000,8.664922,7.739897,5.110477
max,81000.000000,41667.000000,700.000000,480.000000,1.000000,11.302217,10.637489,6.552508


In [13]:
bins=[0,2877.50,3812.50,5795.00,81000.00]
labels=["low","medium","high","very high"]
data["new_income"]=pd.cut(data["Applicant_log"],bins=bins,labels=labels)

In [14]:
bins=[0,100.25,129.00,164.75,700.00]
labels=["low","medium","high","very high"]
data["loan"]=pd.cut(data["LoanAmount_log"],bins=bins,labels=labels)

In [15]:
data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Applicant_log,CoapplicantIncome_log,LoanAmount_log,new_income,loan
0,Male,No,0,Graduate,No,5849.0,0.0,146.412162,360.0,1.0,Urban,yes,8.674197,0.000000,4.993232,low,low
1,Male,Yes,1,Graduate,No,4583.0,1508.0,128.000000,360.0,1.0,Rural,no,8.430327,7.319202,4.859812,low,low
2,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.000000,360.0,1.0,Urban,yes,8.006701,0.000000,4.204693,low,low
3,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.000000,360.0,1.0,Urban,yes,7.857094,7.765993,4.795791,low,low
4,Male,No,0,Graduate,No,6000.0,0.0,141.000000,360.0,1.0,Urban,yes,8.699681,0.000000,4.955827,low,low
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900.0,0.0,71.000000,360.0,1.0,Rural,yes,7.972811,0.000000,4.276666,low,low
610,Male,Yes,3,Graduate,No,4106.0,0.0,40.000000,180.0,1.0,Rural,yes,8.320448,0.000000,3.713572,low,low
611,Male,Yes,1,Graduate,No,8072.0,240.0,253.000000,360.0,1.0,Urban,yes,8.996280,5.484797,5.537334,low,low
612,Male,Yes,2,Graduate,No,7583.0,0.0,187.000000,360.0,1.0,Urban,yes,8.933796,0.000000,5.236442,low,low


In [16]:
# lets create another new column EMI

In [17]:
data["EMI"]=data["LoanAmount"]/data["Loan_Amount_Term"]
data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Applicant_log,CoapplicantIncome_log,LoanAmount_log,new_income,loan,EMI
0,Male,No,0,Graduate,No,5849.0,0.0,146.412162,360.0,1.0,Urban,yes,8.674197,0.000000,4.993232,low,low,0.406700
1,Male,Yes,1,Graduate,No,4583.0,1508.0,128.000000,360.0,1.0,Rural,no,8.430327,7.319202,4.859812,low,low,0.355556
2,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.000000,360.0,1.0,Urban,yes,8.006701,0.000000,4.204693,low,low,0.183333
3,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.000000,360.0,1.0,Urban,yes,7.857094,7.765993,4.795791,low,low,0.333333
4,Male,No,0,Graduate,No,6000.0,0.0,141.000000,360.0,1.0,Urban,yes,8.699681,0.000000,4.955827,low,low,0.391667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900.0,0.0,71.000000,360.0,1.0,Rural,yes,7.972811,0.000000,4.276666,low,low,0.197222
610,Male,Yes,3,Graduate,No,4106.0,0.0,40.000000,180.0,1.0,Rural,yes,8.320448,0.000000,3.713572,low,low,0.222222
611,Male,Yes,1,Graduate,No,8072.0,240.0,253.000000,360.0,1.0,Urban,yes,8.996280,5.484797,5.537334,low,low,0.702778
612,Male,Yes,2,Graduate,No,7583.0,0.0,187.000000,360.0,1.0,Urban,yes,8.933796,0.000000,5.236442,low,low,0.519444


# 1 outlier detection and handling

In [18]:
data.describe()

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Applicant_log,CoapplicantIncome_log,LoanAmount_log,EMI
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,5403.459283,1621.245798,146.412162,342.000000,0.842199,8.341510,4.290129,4.870872,0.476115
std,6109.041673,2926.248369,84.037468,64.372489,0.349681,0.645035,3.875873,0.491376,0.510965
min,150.000000,0.000000,9.000000,12.000000,0.000000,5.017280,0.000000,2.302585,0.025000
25%,2877.500000,0.000000,100.250000,360.000000,1.000000,7.965024,0.000000,4.617584,0.288889
50%,3812.500000,1188.500000,129.000000,360.000000,1.000000,8.246303,7.081125,4.867534,0.369444
75%,5795.000000,2297.250000,164.750000,360.000000,1.000000,8.664922,7.739897,5.110477,0.506389
max,81000.000000,41667.000000,700.000000,480.000000,1.000000,11.302217,10.637489,6.552508,9.250000


In [19]:

def outlier(column):
    q1=data[column].quantile(.25)
    q3=data[column].quantile(.75)
    iqr=q3-q1
    lower_limit=q1-1.5*iqr
    upper_limit=q3+1.5*iqr
    outliers=data[(data[column]<lower_limit)|(data[column]>upper_limit)]
    return q3,q1,iqr,lower_limit,upper_limit,outliers

In [20]:
q3,q1,iqr,lower_limit,upper_limit,outliers=outlier("CoapplicantIncome")

In [21]:
#now lets handle outliers by capping

In [22]:
def capping(column):
    q1=data[column].quantile(.25)
    q3=data[column].quantile(.75)
    iqr=q3-q1
    lower_limit=q1-1.5*iqr
    upper_limit=q3+1.5*iqr
    data[column]=np.where( data[column]<lower_limit,q1,np.where(data[column]>upper_limit,q3,data[column]))
    return data[column]

In [23]:
data["CoapplicantIncome"]=capping("CoapplicantIncome")

In [24]:
q3,q1,iqr,lower_limit,upper_limit,outliers=outlier("LoanAmount")

In [25]:
data["LoanAmount"]=capping("LoanAmount")

In [26]:
q3,q1,iqr,lower_limit,upper_limit,outliers=outlier("Credit_History")
data["Credit_History"]=capping("Credit_History")


In [27]:
q3,q1,iqr,lower_limit,upper_limit,outliers=outlier("Credit_History")

In [28]:
data

,Gender,Married,Dependents,Education,Self_Employed,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Property_Area,Loan_Status,Applicant_log,CoapplicantIncome_log,LoanAmount_log,new_income,loan,EMI
0,Male,No,0,Graduate,No,5849.0,0.0,146.412162,360.0,1.0,Urban,yes,8.674197,0.000000,4.993232,low,low,0.406700
1,Male,Yes,1,Graduate,No,4583.0,1508.0,128.000000,360.0,1.0,Rural,no,8.430327,7.319202,4.859812,low,low,0.355556
2,Male,Yes,0,Graduate,Yes,3000.0,0.0,66.000000,360.0,1.0,Urban,yes,8.006701,0.000000,4.204693,low,low,0.183333
3,Male,Yes,0,Not Graduate,No,2583.0,2358.0,120.000000,360.0,1.0,Urban,yes,7.857094,7.765993,4.795791,low,low,0.333333
4,Male,No,0,Graduate,No,6000.0,0.0,141.000000,360.0,1.0,Urban,yes,8.699681,0.000000,4.955827,low,low,0.391667
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,Female,No,0,Graduate,No,2900.0,0.0,71.000000,360.0,1.0,Rural,yes,7.972811,0.000000,4.276666,low,low,0.197222
610,Male,Yes,3,Graduate,No,4106.0,0.0,40.000000,180.0,1.0,Rural,yes,8.320448,0.000000,3.713572,low,low,0.222222
611,Male,Yes,1,Graduate,No,8072.0,240.0,253.000000,360.0,1.0,Urban,yes,8.996280,5.484797,5.537334,low,low,0.702778
612,Male,Yes,2,Graduate,No,7583.0,0.0,187.000000,360.0,1.0,Urban,yes,8.933796,0.000000,5.236442,low,low,0.519444


In [29]:
preprocessed_data_model_B=data.to_csv("preprocessed_data_model_B.csv")